In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 200
%config InlineBackend.figure_format = 'svg'
# plt.rcParams['font.size'] = 15

In [2]:
def get_vacuum(fname):
    try:
        f = open(fname,"r")
    except:
        raise FileNotFoundError(fname)
    line = f.readline()
    while line != '':
        if 'vacuum' in line:
            vacuum = float(line.split()[-2])
            return vacuum
        else:
            line = f.readline()
def get_eqp_homo_lumo_gap(fname):
    try:
        f = open(fname,"r")
    except:
        raise FileNotFoundError(fname)  
    f.readline();
    f.readline();
    line = f.readline()
    homo = float(line.split()[5])
    line = f.readline()
    lumo = float(line.split()[5])
    return homo,lumo,lumo-homo
def model_func(x,a,b):
    return a+b/x

In [3]:
moles = ['C2H2', 'C2H4', 'C4H4S', 'C6H6', 'CH3Cl', 
        'CH3OH', 'CH3SH', 'CH4', 'Cl2', 'ClF', 'CO', 
        'CO2', 'CS', 'F2', 'H2CO', 'H2O', 'H2O2', 
        'HCl', 'HCN','Na2'] 
refVIP = {'C2H2':11.10, 'C2H4':10.35, 'C4H4S':8.90, 'C6H6':9.10,
          'CH3Cl':11.27, 'CH3OH':10.47, 'CH3SH':9.31, 'CH4':13.99,
          'Cl2':11.48, 'ClF':12.47, 'CO':13.45, 'CO2':13.31,
          'CS':10.92, 'F2':14.90, 'H2CO':10.38, 'H2O':11.81,
          'H2O2':10.96, 'HCl':12.54, 'HCN':13.30, 'Na2':4.73}
refVEA = {'C2H2':-2.495, 'C2H4':-1.798, 'C4H4S':np.nan, 'C6H6':-0.930,
          'CH3Cl':np.nan, 'CH3OH':-0.909, 'CH3SH':np.nan, 'CH4':-0.761,
          'Cl2':1.381, 'ClF':np.nan, 'CO':-0.438, 'CO2':-0.974,
          'CS':0.495, 'F2':1.059, 'H2CO':-0.764, 'H2O':-0.911,
          'H2O2':-1.796, 'HCl':-1.092, 'HCN':-2.250, 'Na2':0.613}

## Read computed GW quasiparticles

In [4]:
eqp = []
for m in moles:
    vacuum = get_vacuum("./G2_97/%s-PBE/pw/%s.out"%(m,m.lower()))
    prefix = "./G2_97/%s-PBE/wfreq"%m
    for i in range(100,401,100):
        fname = prefix+"/kin-20/kin-20-%d/o-eqp.converged.tab"%i
        homo,lumo,gap = get_eqp_homo_lumo_gap(fname)
        eqp.append((m,20,i,homo-vacuum,lumo-vacuum,gap))
    for i in range(100,401,100):
        fname = prefix+"/kin-100/kin-100-%d/o-eqp.converged.tab"%i
        homo,lumo,gap = get_eqp_homo_lumo_gap(fname)
        eqp.append((m,100,i,homo-vacuum,lumo-vacuum,gap))
    for i in range(200,501,100):
        fname = prefix+"/std-%d/o-eqp.converged.tab"%i
        homo,lumo,gap = get_eqp_homo_lumo_gap(fname)
        eqp.append((m,i,0,homo-vacuum,lumo-vacuum,gap))

#     for i in range(120,500,100):
#         fname = prefix+"/std-%d/o-eqp.converged.tab"%i
#         homo,lumo,gap = get_eqp_homo_lumo_gap(fname)
#         eqp.append((m,i,0,homo-vacuum,lumo-vacuum,gap))


# print(eqp)
eqp = np.array(eqp,dtype=[('mol','<U32'),('std',int),('kin',int),('homo',float),('lumo',float),('gap',float)])
eqp_df = pd.DataFrame(eqp,columns=['mol','std','kin','homo','lumo','gap'])
# eqp_df

## Interpolate the quasiparticle energies

In [5]:
eqp_interp = []
for i in range(0,len(eqp),4):
    mol = eqp['mol'][i]
    tmpstd = eqp['std'][i:i+4]
    tmpkin = eqp['kin'][i:i+4]
    tmphomo = eqp['homo'][i:i+4]
    tmplumo = eqp['lumo'][i:i+4]
    popt,popv = curve_fit(model_func,tmpstd+tmpkin,tmphomo)
    homo = popt[0]
    popt,popv = curve_fit(model_func,tmpstd+tmpkin,tmplumo)
    lumo = popt[0]
    eqp_interp.append((mol,homo,lumo,lumo-homo))

In [6]:
# Output interpolated HOMO
for i in range(0,len(eqp_interp),3):
    # print order : kin20, kin100, std100, without ref
    print("%6s & %6.2f & %6.2f & %6.2f \\\\"%(moles[i//3],-eqp_interp[i][1],-eqp_interp[i+1][1],-eqp_interp[i+2][1]))

  C2H2 &  11.07 &  11.06 &  11.06 \\
  C2H4 &  10.41 &  10.40 &  10.40 \\
 C4H4S &   8.80 &   8.77 &   8.76 \\
  C6H6 &   9.17 &   9.14 &   9.13 \\
 CH3Cl &  11.28 &  11.26 &  11.25 \\
 CH3OH &  10.58 &  10.56 &  10.56 \\
 CH3SH &   9.39 &   9.36 &   9.36 \\
   CH4 &  14.01 &  14.01 &  14.01 \\
   Cl2 &  11.51 &  11.51 &  11.50 \\
   ClF &  12.55 &  12.55 &  12.54 \\
    CO &  13.51 &  13.50 &  13.50 \\
   CO2 &  13.32 &  13.31 &  13.31 \\
    CS &  11.00 &  10.98 &  10.98 \\
    F2 &  14.99 &  14.97 &  14.97 \\
  H2CO &  10.43 &  10.42 &  10.42 \\
   H2O &  11.82 &  11.82 &  11.81 \\
  H2O2 &  10.87 &  10.87 &  10.86 \\
   HCl &  12.50 &  12.50 &  12.50 \\
   HCN &  13.20 &  13.20 &  13.20 \\
   Na2 &   4.95 &   4.95 &   4.95 \\


In [7]:
# Output interpolated LUMO
for i in range(0,len(eqp_interp),3):
    # print order : kin20, kin100, std100, without ref
    print("%6s & %6.2f & %6.2f & %6.2f \\\\"%(moles[i//3],-eqp_interp[i][2],-eqp_interp[i+1][2],-eqp_interp[i+2][2]))

  C2H2 &  -2.42 &  -2.41 &  -2.41 \\
  C2H4 &  -1.75 &  -1.75 &  -1.75 \\
 C4H4S &  -0.85 &  -0.81 &  -0.80 \\
  C6H6 &  -1.01 &  -0.96 &  -0.96 \\
 CH3Cl &  -1.17 &  -1.16 &  -1.16 \\
 CH3OH &  -0.89 &  -0.89 &  -0.89 \\
 CH3SH &  -0.88 &  -0.88 &  -0.88 \\
   CH4 &  -0.64 &  -0.64 &  -0.64 \\
   Cl2 &   1.65 &   1.64 &   1.65 \\
   ClF &   1.28 &   1.28 &   1.28 \\
    CO &  -1.56 &  -1.57 &  -1.57 \\
   CO2 &  -0.97 &  -0.97 &  -0.97 \\
    CS &   0.49 &   0.51 &   0.51 \\
    F2 &   1.16 &   1.16 &   1.16 \\
  H2CO &  -0.69 &  -0.68 &  -0.68 \\
   H2O &  -0.90 &  -0.90 &  -0.90 \\
  H2O2 &  -1.80 &  -1.79 &  -1.79 \\
   HCl &  -1.07 &  -1.07 &  -1.07 \\
   HCN &  -2.08 &  -2.08 &  -2.08 \\
   Na2 &   0.64 &   0.63 &   0.63 \\
